# wps_parameters

#### wps_parameters is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools.testing import get_target_url
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("osprey")
print(f"Using osprey on {url}")

Using osprey on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps


In [3]:
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
osprey.parameters?

Signature:
osprey.parameters(
    case_id,
    grid_id,
    pour_points=None,
    uh_box=None,
    routing=None,
    domain=None,
    loglevel='INFO',
    np=1,
    version=True,
    params_config_file=None,
    params_config_dict=None,
)
Docstring:
Develop impulse response functions using inputs from a configuration file or dictionary

Parameters
----------
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
np : integer
    Number of processors used to run job
version : boolean
    Return RVIC version string
case_id : string
    Case ID for the RVIC process
grid_id : string
    Routing domain grid shortname
pour_points : ComplexData:mimetype:`text/plain`, :mimetype:`text/csv`
    Path to Pour Points File; A comma separated file of outlets to route to [lons, lats]
uh_box : ComplexData:mimetype:`text/plain`, :mimetype:`text/csv`
    Path to UH Box File. This defines the unit hydrograph to rout flow to the edge of each grid cell.
routing : Comp

#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
# run parameters only with required inputs
output_basic = osprey.parameters(
    case_id = "sample",
    grid_id = "COLUMBIA",
    pour_points = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_pour.txt",
    uh_box = resource_filename("tests", "/data/samples/uhbox.csv"),
    routing = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_flow_parameters.nc",
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
)

In [6]:
# run parameters with local configuration file
output_config_file = osprey.parameters(
    case_id = "sample",
    grid_id = "COLUMBIA",
    pour_points = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_pour.txt",
    uh_box = resource_filename("tests", "/data/samples/uhbox.csv"),
    routing = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_flow_parameters.nc",
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
    params_config_file = resource_filename("tests", "/data/configs/parameters.cfg")
)

In [7]:
# run parameters with configuration dictionary
output_config_dict = osprey.parameters(
    case_id = "sample",
    grid_id = "COLUMBIA",
    pour_points = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_pour.txt",
    uh_box = resource_filename("tests", "/data/samples/uhbox.csv"),
    routing = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_flow_parameters.nc",
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
    params_config_dict = {
        "OPTIONS": {
            "LOG_LEVEL": "CRITICAL",
        },
    }
)

Access the output with nc_to_dataset() or auto_construct_outputs() from wps_tools.output_handling

In [8]:
# NBVAL_IGNORE_OUTPUT
from wps_tools.output_handling import nc_to_dataset, auto_construct_outputs

# Example parameters only with required inputs
nc_to_dataset(output_basic.get()[0])
auto_construct_outputs(output_basic.get())

[<class 'netCDF4._netCDF4.Dataset'>
 root group (NETCDF4 data model, file format HDF5):
     title: RVIC parameter file
     comment: Output from the RVIC Streamflow Routing Model.
     Conventions: CF-1.6
     history: Created: Wed Jan  6 17:16:46 2021
     source: /tmp/osprey-venv/bin/osprey
     institution: University of Washington
     hostname: pcic-2003
     username: csanders
     references: Based on the initial model of Lohmann, et al., 1996, Tellus, 48(A), 708-721
     version: 1.1.4
     RvicPourPointsFile: sample_pour.txt
     RvicUHFile: uhbox.csv
     RvicFdrFile: sample_flow_parameters.nc
     RvicDomainFile: sample_routing_domain.nc
     featureType: timeSeries
     dimensions(sizes): timesteps(100), outlets(1), nc_chars(256), sources(1), tracers(1)
     variables(dimensions): float64 timesteps(timesteps), int32 full_time_length(), int32 subset_length(), float64 unit_hydrograph_dt(), float64 outlet_lon(outlets), float64 outlet_lat(outlets), int32 outlet_x_ind(outlets),

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [9]:
input_config = read_config("tests/data/configs/parameters.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": "sample_pour.txt",
    "RvicFdrFile": "sample_flow_parameters.nc",
}

In [10]:
def test_parameters_output(output, expected):
    output_data = nc_to_dataset(output.get()[0])

    # Metadata of the output of RVIC parameters module
    metadata = {
        "title": output_data.title,
        "year": output_data.history.split()[5],
        "month": output_data.history.split()[2],
        "day": output_data.history.split()[3].zfill(2),
        "RvicPourPointsFile": output_data.RvicPourPointsFile,
        "RvicFdrFile": output_data.RvicFdrFile,
    }

    assert metadata == expected

In [11]:
test_parameters_output(output_basic, expected)
test_parameters_output(output_config_file, expected)
test_parameters_output(output_config_dict, expected)